In [1]:
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
path = "/content/drive/My Drive/Khoa Luan/"

In [3]:
!unzip -q "/content/drive/My Drive/Khoa Luan/VLSP_DATA/VLSP_2018.zip"

unzip:  cannot find or open /content/drive/My Drive/Khoa Luan/VLSP_DATA/VLSP_2018.zip, /content/drive/My Drive/Khoa Luan/VLSP_DATA/VLSP_2018.zip.zip or /content/drive/My Drive/Khoa Luan/VLSP_DATA/VLSP_2018.zip.ZIP.


In [4]:
from sklearn.datasets import load_files
review_train = load_files('/content/Food/data_train/Train',encoding='utf-8').data
label_train = load_files('/content/Food/data_train/Train').target.tolist()
review_test = load_files('/content/Food/data_test/Test',encoding='utf-8').data
label_test = load_files('/content/Food/data_test/Test').target.tolist()

FileNotFoundError: ignored

In [ ]:
!pip3 install vncorenlp
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

In [ ]:
from vncorenlp import VnCoreNLP
tokenizer = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [ ]:
import pickle 
replace_list = pickle.load(open(path+'replace_list.pkl','rb'))

In [ ]:
import re
from gensim.utils import simple_preprocess
import pandas as pd
from nltk import flatten

In [ ]:
VN_CHARS_LOWER = u'ạảãàáâậầấẩẫăắằặẳẵóòọõỏôộổỗồốơờớợởỡéèẻẹẽêếềệểễúùụủũưựữửừứíìịỉĩýỳỷỵỹđð'
VN_CHARS_UPPER = u'ẠẢÃÀÁÂẬẦẤẨẪĂẮẰẶẲẴÓÒỌÕỎÔỘỔỖỒỐƠỜỚỢỞỠÉÈẺẸẼÊẾỀỆỂỄÚÙỤỦŨƯỰỮỬỪỨÍÌỊỈĨÝỲỶỴỸÐĐ'
VN_CHARS = VN_CHARS_LOWER + VN_CHARS_UPPER
def no_marks(s):
    __INTAB = [ch for ch in VN_CHARS]
    __OUTTAB = "a"*17 + "o"*17 + "e"*11 + "u"*11 + "i"*5 + "y"*5 + "d"*2
    __OUTTAB += "A"*17 + "O"*17 + "E"*11 + "U"*11 + "I"*5 + "Y"*5 + "D"*2
    __r = re.compile("|".join(__INTAB))
    __replaces_dict = dict(zip(__INTAB, __OUTTAB))
    result = __r.sub(lambda m: __replaces_dict[m.group(0)], s)
    return result

In [ ]:
with open(path+'pos.txt','r') as f:
    Pos_list = [i.strip() for i in f]

In [ ]:
with open(path+'neg.txt', 'r') as f:
    Neg_list = [i.strip() for i in f]

In [ ]:
def preprocess(data):
    token = []
    token_nomarks = []
    for text in data:
        check = re.search(r'([a-z])\1+',text)
        if check:
          if len(check.group())>2:
            text = re.sub(r'([a-z])\1+', lambda m: m.group(1), text, flags=re.IGNORECASE) #remove các ký tự kéo dài như hayyy,ngonnnn...

        text = text.strip() #loại dấu cách đầu câu
        text = text.lower() #chuyển tất cả thành chữ thường

        text = re.sub('< a class.+</a>',' ',text)

        for k, v in replace_list.items():       #replace các từ có trong replace_list
          text = text.replace(k, v)       

        text = re.sub(r'_',' ',text)  
        
        text = ' '.join(i for i in flatten(tokenizer.tokenize(text)))             #gán từ ghép
        
        for i in Pos_list:                                    #thêm feature positive
          if re.search(' '+i+' ',text): 
            text = re.sub(i,i+' positive ',text)
        for i in Neg_list:                                    #thêm feature negative
          if re.search(' '+i+' ',text):
            text = re.sub(i,i+' negative ',text)
        
        text_nomarks = no_marks(re.sub(r'_',' ',text))        #lấy câu không dấu

        text = ' '.join(i for i in simple_preprocess(text))
        text_nomarks = ' '.join(i for i in simple_preprocess(text_nomarks))

        token.append(text)
        token_nomarks.append(text_nomarks)
    return token,token_nomarks

In [ ]:
def no_preprocess(data):
  token = []
  token_nomarks = []
  for text in data:
    text = ' '.join(i for i in flatten(tokenizer.tokenize(text)))
    text_nomarks = no_marks(re.sub(r'_',' ',text))
    token.append(text)
    token_nomarks.append(text_nomarks)
  return token,token_nomarks

In [ ]:
# @title VLSP PREPROCESS

In [ ]:
vlsp,vlsp_nomarks = preprocess(review_train)

In [ ]:
for i in Pos_list:
  vlsp.append(i)
  label_train.append(1)
for i in Neg_list:
  vlsp.append(i)
  label_train.append(0)

In [ ]:
df = pd.DataFrame(list(zip(vlsp,label_train)),columns=['review','label'])

In [ ]:
df.to_csv(path+'VLSP_DATA/VLSP_train7.csv')

In [ ]:
vlsp_test,_ = preprocess(review_test)

In [ ]:
df_test = pd.DataFrame(list(zip(vlsp_test,label_test)),columns=['review','label'])
df_test.to_csv(path+'VLSP_DATA/VLSP_test7.csv')

In [ ]:
# @title AIVIVN PREPROCESS

In [ ]:
df_aivivn = pd.read_csv(path+'AIVIVN/AIVIVN_train.csv')
df_aivivn = df_aivivn.dropna()
aivivn_train = df_aivivn.review.tolist()
label_aivivn = df_aivivn.label.tolist()

In [ ]:
#aivivn,aivivn_nomarks = no_preprocess(aivivn_train)
aivivn,aivivn_nomarks = preprocess(aivivn_train)

In [ ]:
aivivn+=aivivn_nomarks
label_aivivn+=label_aivivn

In [ ]:
for i in Pos_list:
  aivivn.append(i)
  label_train.append(1)
for i in Neg_list:
  aivivn.append(i)
  label_train.append(0)

In [ ]:
df_aivivn = pd.DataFrame(list(zip(aivivn,label_train)),columns=['review','label'])
#df_aivivn.to_csv(path+'AIVIVN/preprocess_train.csv')

In [ ]:
#@title CONCAT 2 DATA

In [ ]:
from sklearn.model_selection import train_test_split
aivivn_train,aivivn_test,label_aivivn_train,label_aivivn_test = train_test_split(aivivn,label_aivivn,test_size=0.3,random_state=48)

In [ ]:
vlsp+=aivivn_train
label_train+=label_aivivn_train

In [ ]:
df = pd.DataFrame(list(zip(vlsp,label_train)),columns=['review','label'])

In [ ]:
df.to_csv(path+'data1.csv')

In [ ]:
vlsp_test+=aivivn_test
label_test+=label_aivivn_test

In [ ]:
df_test = pd.DataFrame(list(zip(vlsp_test,label_test)),columns=['review','label'])
df_test.to_csv(path+'test1.csv')